In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import math
import category_encoders as ce

<IPython.core.display.Javascript object>

In [3]:
# Load original dataset from csv file
# Drop all nan value
# Drop unnamed columns
# Drop address
# Drop sale announcement date
df = pd.read_csv("VN_housing_dataset.csv")
df = df.drop(columns=["Unnamed: 0", "Ngày", "Địa chỉ"])
df = df.dropna()

# Filter out price (million/m2)
df = df[df["Giá/m2"].str.contains("triệu/m²")]
df["Giá/m2"] = [float(x.replace(",", ".").split()[0]) for x in list(df["Giá/m2"])]

# Filter out area(s), length(l), width(w)
# Drop out all instances i such that: l*w*(1-eps_s) > s or l*w*(1+eps_s) < s
eps_s = 0.05
df["Diện tích"] = [float(x.split()[0]) for x in list(df["Diện tích"])]
df["Dài"] = [float(x.split()[0]) for x in list(df["Dài"])]
df["Rộng"] = [float(x.split()[0]) for x in list(df["Rộng"])]
lower = df[df["Dài"] * df["Rộng"] * (1 - eps_s) > df["Diện tích"]].index
df = df.drop(lower)
upper = df[df["Dài"] * df["Rộng"] * (1 + eps_s) < df["Diện tích"]].index
df = df.drop(upper)

# Filter out number of bedrooms (nb)
# Drop out all instances i such that: nb >= 10
df = df.drop(df[df["Số phòng ngủ"] == "nhiều hơn 10 phòng"].index)
df["Số phòng ngủ"] = [float(x.split()[0]) for x in list(df["Số phòng ngủ"])]

# Filter out number of floor (nf)
# Drop out all instances i such that: nf >= 10
df = df.drop(df[df["Số tầng"] == "Nhiều hơn 10"].index)
df["Số tầng"] = [float(x) for x in list(df["Số tầng"])]
df = df.drop(df[df["Số tầng"] > 10].index)

print(f"{len(df)} instances")
display(df.head())

6647 instances


,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
2,Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,10.0,4.0,65.00
15,Quận Long Biên,Phường Bồ Đề,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,52.0,12.0,4.2,93.27
24,Quận Hà Đông,Phường La Khê,"Nhà mặt phố, mặt tiền",Đã có sổ,5.0,5.0,90.0,18.0,5.0,108.89
34,Huyện Thanh Trì,Xã Tả Thanh Oai,"Nhà ngõ, hẻm",Đã có sổ,3.0,2.0,42.0,11.0,4.0,29.76
37,Huyện Thanh Trì,Xã Tam Hiệp,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,43.0,11.0,4.0,50.00


<IPython.core.display.Javascript object>

In [4]:
def review(df, attribute):
    print("=" * 100)
    print(set(list(df[attribute])))
    for x in set(list(df[attribute])):
        print(f"{x}: {list(df[attribute]).count(x)}")

<IPython.core.display.Javascript object>

In [5]:
review(df, "Giấy tờ pháp lý")
review(df, "Loại hình nhà ở")
review(df, "Huyện")
review(df, "Quận")

{'Đang chờ sổ', 'Giấy tờ khác', 'Đã có sổ'}
Đang chờ sổ: 53
Giấy tờ khác: 44
Đã có sổ: 6550
{'Nhà ngõ, hẻm', 'Nhà phố liền kề', 'Nhà mặt phố, mặt tiền', 'Nhà biệt thự'}
Nhà ngõ, hẻm: 4676
Nhà phố liền kề: 254
Nhà mặt phố, mặt tiền: 1587
Nhà biệt thự: 130
{'Phường Bồ Đề', 'Phường Phương Mai', 'Xã Đông La', 'Phường Thành Công', 'Phường Cổ Nhuế 2', 'Xã Tân Lập', 'Xã Kiêu Kỵ', 'Phường Cửa Đông', 'Xã Cổ Đông', 'Phường Dương Nội', 'Xã Thanh Liệt', 'Phường Nhật Tân', 'Xã Mai Lâm', 'Phường Đồng Tâm', 'Phường Thụy Phương', 'Phường Tân Mai', 'Xã Liên Ninh', 'Phường Quán Thánh', 'Phường Bách Khoa', 'Phường Thanh Trì', 'Xã Tam Hiệp', 'Thị trấn Văn Điển', 'Phường Thanh Xuân Nam', 'Phường Láng Hạ', 'Phường Thượng Đình', 'Xã Thạch Hoà', 'Phường Khương Mai', 'Phường Văn Quán', 'Phường Ngã Tư Sở', 'Phường Thanh Xuân Trung', 'Phường Cát Linh', 'Phường Giảng Võ', 'Xã Vân Nội', 'Phường Khương Đình', 'Phường Láng Thượng', 'Phường Vạn Phúc', 'Xã Ngọc Hồi', 'Phường Quỳnh Mai', 'Phường Nguyễn Trung Trực', 'Ph

<IPython.core.display.Javascript object>

In [6]:
# Rename columns
df = df.reset_index(drop=True)
df = df.rename(
    columns={
        "Quận": "district",
        "Huyện": "ward",
        "Loại hình nhà ở": "type_of_housing",
        "Giấy tờ pháp lý": "legal_paper",
        "Số tầng": "floor_num",
        "Số phòng ngủ": "bedroom_num",
        "Diện tích": "area",
        "Dài": "length",
        "Rộng": "width",
        "Giá/m2": "price/m2",
    }
)
print(f"{len(df)} instances")
display(df.head())

6647 instances


,district,ward,type_of_housing,legal_paper,floor_num,bedroom_num,area,length,width,price/m2
0,Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,10.0,4.0,65.00
1,Quận Long Biên,Phường Bồ Đề,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,52.0,12.0,4.2,93.27
2,Quận Hà Đông,Phường La Khê,"Nhà mặt phố, mặt tiền",Đã có sổ,5.0,5.0,90.0,18.0,5.0,108.89
3,Huyện Thanh Trì,Xã Tả Thanh Oai,"Nhà ngõ, hẻm",Đã có sổ,3.0,2.0,42.0,11.0,4.0,29.76
4,Huyện Thanh Trì,Xã Tam Hiệp,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,43.0,11.0,4.0,50.00


<IPython.core.display.Javascript object>

In [7]:
# Ordinal encoding: type_of_housing, legal_paper
df["type_of_housing"] = df.type_of_housing.map(
    {
        "Nhà biệt thự": 4.0,
        "Nhà mặt phố, mặt tiền": 3.0,
        "Nhà phố liền kề": 2.0,
        "Nhà ngõ, hẻm": 1.0,
    }
)
df["legal_paper"] = df.legal_paper.map(
    {"Đã có sổ": 3.0, "Đang chờ sổ": 2.0, "Giấy tờ khác": 1.0}
)

print(f"{len(df)} instances")
display(df.head())

6647 instances


,district,ward,type_of_housing,legal_paper,floor_num,bedroom_num,area,length,width,price/m2
0,Quận Hai Bà Trưng,Phường Minh Khai,1.0,3.0,4.0,4.0,40.0,10.0,4.0,65.00
1,Quận Long Biên,Phường Bồ Đề,1.0,3.0,5.0,4.0,52.0,12.0,4.2,93.27
2,Quận Hà Đông,Phường La Khê,3.0,3.0,5.0,5.0,90.0,18.0,5.0,108.89
3,Huyện Thanh Trì,Xã Tả Thanh Oai,1.0,3.0,3.0,2.0,42.0,11.0,4.0,29.76
4,Huyện Thanh Trì,Xã Tam Hiệp,1.0,3.0,4.0,4.0,43.0,11.0,4.0,50.00


<IPython.core.display.Javascript object>

In [8]:
# Nominal encoding: district, ward
df = ce.OneHotEncoder(cols=["district", "ward"]).fit_transform(df)
print(f"{len(df)} instances")
display(df.head())

6647 instances


,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,...,ward_219,ward_220,type_of_housing,legal_paper,floor_num,bedroom_num,area,length,width,price/m2
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1.0,3.0,4.0,4.0,40.0,10.0,4.0,65.00
1,0,1,0,0,0,0,0,0,0,0,...,0,0,1.0,3.0,5.0,4.0,52.0,12.0,4.2,93.27
2,0,0,1,0,0,0,0,0,0,0,...,0,0,3.0,3.0,5.0,5.0,90.0,18.0,5.0,108.89
3,0,0,0,1,0,0,0,0,0,0,...,0,0,1.0,3.0,3.0,2.0,42.0,11.0,4.0,29.76
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1.0,3.0,4.0,4.0,43.0,11.0,4.0,50.00


<IPython.core.display.Javascript object>